In [1]:
from pathlib import Path
from time import perf_counter
import sys

sys.path.insert(0, str(Path().absolute().parent))

In [2]:
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score

In [3]:
from settings import SETTINGS, PATH
from catboost_approach import MyCB, TARGET_FEATURE
from utils import read, write

In [4]:
# %%time
# model = MyCB()
# model.read_data(force=True)
# raise ValueError

In [5]:
common_param = {'verbose': 0, 'random_state': SETTINGS['random_seed']}
forbidden_cols = ['verbose', 'random_state', 'res', 'time']
if PATH['catboost_tune_pars'].exists():
    df = read(PATH['catboost_tune_pars'])
else:
    df = pd.DataFrame()
pars = [df]

model = MyCB(edge=20, **common_param)
model.read_data()
x, y = model.my_data

def objective(trial, x=x, y=y, common_param=common_param, forbidden_cols=forbidden_cols, pars=pars):
    t = perf_counter()
    param = {'objective': trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
             'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.01, 0.12),
             'depth': trial.suggest_int('depth', 4, 12),
             'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
             'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS'])}
    param['iterations'] = trial.suggest_int('iterations', 500, 2000)
    # param['grow_policy'] =  trial.suggest_categorical('grow_policy', ['Lossguide', 'SymmetricTree', 'Depthwise'])
    param['leaf_estimation_backtracking'] =  trial.suggest_categorical('leaf_estimation_backtracking', ['AnyImprovement', 'No'])
    param['l2_leaf_reg'] = trial.suggest_float('l2_leaf_reg', 0.1, 20)

    if param['bootstrap_type'] == 'Bayesian':
        param['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif param['bootstrap_type'] == 'Bernoulli':
        param['subsample'] = trial.suggest_float('subsample', 0.1, 1)
    trial.set_user_attr('param', param)
    trial.set_user_attr('common_param', common_param)
    
    sub_df = pars[0].copy()
    for col, val in (common_param | param).items():
        if col not in forbidden_cols and sub_df.shape[0] > 0:
            sub_df = sub_df[sub_df[col] == val]
    if sub_df.shape[0] == 1:
        print('trial was calculated earlier')
        return np.mean(sub_df.iloc[0]['res'])
    elif sub_df.shape[0] > 1:
        raise ValueError('there are so many results of trials')
    
    model = MyCB(**(common_param | param))
    res = cross_val_score(model, x, y, cv=3, verbose=0)
    
    pars[0] = pars[0].append(common_param | param | {'res': res, 'time': perf_counter() - t}, ignore_index=True)
    print(np.mean(res), perf_counter() - t, 'seconds')
    return np.mean(res)

In [6]:
study = optuna.create_study(sampler=TPESampler(), direction='maximize')
# while True:
for _ in range(100):
    study.optimize(objective, n_trials=1, n_jobs=1)
    write(PATH['catboost_tune_pars'], pars[0])

[I 2022-12-14 09:31:15,940] A new study created in memory with name: no-name-5882d1e9-20c6-4a44-b8f7-f3f7869b0c72
C:\Users\user\AppData\Local\Temp\ipykernel_2684\2085184799.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pars[0] = pars[0].append(common_param | param | {'res': res, 'time': perf_counter() - t}, ignore_index=True)
[I 2022-12-14 09:31:20,831] Trial 0 finished with value: 0.7246462285473741 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.06555764042182215, 'depth': 4, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'iterations': 593, 'leaf_estimation_backtracking': 'AnyImprovement', 'l2_leaf_reg': 1.7560305147604929, 'subsample': 0.5481518625627582}. Best is trial 0 with value: 0.7246462285473741.


0.7246462285473741 4.8906757 seconds


C:\Users\user\AppData\Local\Temp\ipykernel_2684\2085184799.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pars[0] = pars[0].append(common_param | param | {'res': res, 'time': perf_counter() - t}, ignore_index=True)
[I 2022-12-14 09:31:25,394] Trial 1 finished with value: 0.7055064565528957 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.016891501405372704, 'depth': 7, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'iterations': 924, 'leaf_estimation_backtracking': 'No', 'l2_leaf_reg': 5.170529896958784, 'subsample': 0.99218113703203}. Best is trial 0 with value: 0.7246462285473741.


0.7055064565528957 4.5629095 seconds


C:\Users\user\AppData\Local\Temp\ipykernel_2684\2085184799.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pars[0] = pars[0].append(common_param | param | {'res': res, 'time': perf_counter() - t}, ignore_index=True)
[I 2022-12-14 09:31:34,144] Trial 2 finished with value: 0.7328341757713565 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.1003185466508871, 'depth': 8, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'iterations': 1899, 'leaf_estimation_backtracking': 'No', 'l2_leaf_reg': 18.887576755777264, 'bagging_temperature': 2.7292186064175983}. Best is trial 2 with value: 0.7328341757713565.


...

0.7328436045302983 8.267665799999804 seconds


C:\Users\user\AppData\Local\Temp\ipykernel_2684\2085184799.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pars[0] = pars[0].append(common_param | param | {'res': res, 'time': perf_counter() - t}, ignore_index=True)
[I 2022-12-14 09:52:04,531] Trial 98 finished with value: 0.7328078602739873 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.1092899885599692, 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'iterations': 1781, 'leaf_estimation_backtracking': 'No', 'l2_leaf_reg': 3.06373295953629, 'subsample': 0.13050963253380726}. Best is trial 26 with value: 0.7328795263180226.


0.7328078602739873 8.456056799999942 seconds


C:\Users\user\AppData\Local\Temp\ipykernel_2684\2085184799.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pars[0] = pars[0].append(common_param | param | {'res': res, 'time': perf_counter() - t}, ignore_index=True)
[I 2022-12-14 09:52:13,266] Trial 99 finished with value: 0.7327450988344085 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.07324466496034578, 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'iterations': 1840, 'leaf_estimation_backtracking': 'No', 'l2_leaf_reg': 1.6491650689167847, 'subsample': 0.17904498859689097}. Best is trial 26 with value: 0.7328795263180226.


0.7327450988344085 8.732876199999964 seconds


In [25]:
dfs = dict()
best = []
for file in PATH['catboost_tune_pars'].parent.glob(PATH['catboost_tune_pars'].stem + '*'):
    df = read(file)
    df['res'] = df['res'].apply(np.mean)
    df = df.sort_values('res', ascending=False)
    best.append(df.iloc[0])
    dfs[file.stem.split('_')[-1]] = df

In [28]:
pd.concat(best, axis=1).T

,verbose,random_state,objective,colsample_bylevel,depth,boosting_type,bootstrap_type,iterations,leaf_estimation_backtracking,l2_leaf_reg,res,time,subsample,bagging_temperature
685,0,0,Logloss,0.093803,5,Ordered,Bernoulli,1624,AnyImprovement,15.159738,0.675632,43.689904,0.142875,NaN
79,0,0,Logloss,0.119052,11,Ordered,Bernoulli,1380,No,7.508226,0.732897,18.739581,0.113953,NaN
126,0,0,Logloss,0.114089,5,Ordered,Bernoulli,1975,AnyImprovement,1.379904,0.896922,58.913446,0.134022,NaN
